In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
%reload_ext autoreload
%autoreload 2
from HelperFunctions import feature_standardize, label_encode_column, columns_of_type

from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
reg = RandomForestRegressor()
clf = RandomForestClassifier()

import pickle
from functools import wraps

In [2]:
df=pd.read_csv('Sampled_Dataset_Balanced.csv')
# df=pd.read_csv('down_sampled_df_v2.csv')
temp_pre_df=pd.read_csv('pre_downsample_df.csv')
pre_df=temp_pre_df.sample(50000)

In [3]:
df.head()

,acc_now_delinq,addr_state,all_util,annual_inc,application_type,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,emp_length,funded_amnt,grade,home_ownership,il_util,initial_list_status,inq_last_12m,int_rate,issue_d,mo_sin_rcnt_tl,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,num_accts_ever_120_pd,num_rev_accts,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,num_il_tl,pct_tl_nvr_dlq,pub_rec,pub_rec_bankruptcies,purpose,revol_util,sub_grade,tax_liens,term,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bc_limit,total_il_high_credit_limit,total_rev_hi_lim,verification_status,loan_status,fico_score,active_rev_acct_frac,active_ins_acct_frac,total_payout,duration,return_rate,credit_hist,treasury_rate,unemp_rate_3mon
0,0.0,MO,-999.0,45600.0,Individual,0.0,0.0,1.0,0.0,3.26,10,4650.0,C,MORTGAGE,-999.0,f,-999.0,10.28,2007-10-01,-999.0,22.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,3.0,-999.0,-999.0,0.0,-999.0,other,35.0,C1,0.0,36,-999.0,-999.0,-999.0,25.0,-999.0,-999.0,-999.0,Not Verified,Default,687.0,-999.0,-999.0,3949.29,2.086073,-0.075309,17.926027,4.01,5.20
1,0.0,CO,-999.0,14400.0,Individual,0.0,0.0,0.0,0.0,1.67,1,4000.0,C,RENT,-999.0,f,-999.0,11.22,2007-10-01,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,1.0,-999.0,-999.0,0.0,-999.0,car,37.0,C4,0.0,36,-999.0,-999.0,-999.0,2.0,-999.0,-999.0,-999.0,Not Verified,Fully Paid,672.0,-999.0,-999.0,4730.55,3.253196,0.052916,3.501370,4.01,3.53
2,0.0,FL,-999.0,43500.0,Individual,0.0,0.0,0.0,0.0,3.01,4,4900.0,E,RENT,-999.0,f,-999.0,13.75,2007-10-01,-999.0,0.0,-999.0,63.0,-999.0,-999.0,-999.0,-999.0,5.0,-999.0,-999.0,1.0,-999.0,debt_consolidation,90.1,E2,0.0,36,-999.0,-999.0,-999.0,5.0,-999.0,-999.0,-999.0,Not Verified,Default,647.0,-999.0,-999.0,5693.25,2.918950,0.052748,3.501370,4.01,4.47
3,0.0,FL,-999.0,72000.0,Individual,0.0,0.0,0.0,0.0,17.83,4,10250.0,E,RENT,-999.0,f,-999.0,14.38,2007-10-01,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,28.0,-999.0,-999.0,0.0,-999.0,debt_consolidation,6.5,E4,0.0,36,-999.0,-999.0,-999.0,32.0,-999.0,-999.0,-999.0,Not Verified,Fully Paid,687.0,-999.0,-999.0,11444.37,1.003881,0.116048,3.586301,4.01,4.47
4,0.0,WA,-999.0,120000.0,Individual,0.0,0.0,0.0,0.0,16.97,7,15450.0,C,RENT,-999.0,f,-999.0,10.91,2007-11-01,-999.0,0.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,27.0,-999.0,-999.0,0.0,-999.0,debt_consolidation,44.5,C3,0.0,36,-999.0,-999.0,-999.0,39.0,-999.0,-999.0,-999.0,Not Verified,Default,697.0,-999.0,-999.0,12312.48,4.168265,-0.053002,26.520548,3.35,4.37


In [4]:
drop_list=['sub_grade','issue_d','zip_code','RANDOM','id','weight','loan_duration','loan_status','return_rate']

In [5]:
grade_weight_dict={'A':1,
            'B':1,
            'C':1,
            'D':1,
            'E':1,
            'F':1,
            'G':1}

#### Transformation function that transforms data into ready-train, ready-target dataframes.

In [6]:
def data_transform(df,col_to_drop,target,grade_weight_dict):
    df['weight']=df['grade'].map(grade_weight_dict)
    grade_weight_array=df['weight'].values
    index = df.index
    grade_weight_frame = pd.DataFrame(grade_weight_array, index=index)

    x_train=df.drop(col_to_drop,axis=1)
    y_train=df.loc[:,target]

    cat_list=columns_of_type(x_train,'string')
    cont_list=columns_of_type(x_train,'number')

    x_train.reset_index(drop=True,inplace=True)
    label_encoded_df=label_encode_column(x_train,cat_list)

    #     final_train_df=label_encoded_df.apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
    final_train_df=label_encoded_df
    return final_train_df,y_train

# The Regression Model

In [7]:
def LC_Regressor(df,col_to_drop,target,**grade_weight_dict):
    '''
    df: dataframe of interest (for x_train)
    col_to_drop: accepts a list of column names (strings) that will be dropped from the model
    target: the target variable, in the format of a string
    **grade_weight_dict: the dictionary for weights for different grades
    **grade_weight_dict is in the form of:
    grade_dict={'A':1,
                'B':1,
                'C':1,
                'D':1,
                'E':1,
                'F':1,
                'G':1}
    '''
    df['weight']=df['grade'].map(grade_weight_dict)
    grade_weight_array=df['weight'].values
    index = df.index
    grade_weight_frame = pd.DataFrame(grade_weight_array, index=index)

    x_train=df.drop(col_to_drop,axis=1)
    y_train=df.loc[:,target]
    
    cat_list=columns_of_type(x_train,'string')
    cont_list=columns_of_type(x_train,'number')
    
    x_train.reset_index(drop=True,inplace=True)
    label_encoded_df=label_encode_column(x_train,cat_list)
    
#     final_train_df=label_encoded_df.apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
    final_train_df=label_encoded_df
    
    def grade_weighted_r2(y_true,y_pred,sample_weight):        

        weighted_r2=r2_score(y_true,y_pred,sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1))
        return weighted_r2    

    score_params = {"sample_weight": grade_weight_frame}

    grade_weighted_scorer = make_scorer(score_func=grade_weighted_r2,
                                        greater_is_better=True,
                                        needs_proba=False,
                                        needs_threshold=False,
                                        **score_params)
    grid_para_forest = {
    'n_estimators': [50],
    'max_depth':[None],
    'max_features':['auto'],
    'min_samples_leaf':[1],
    'min_samples_split':[2]
    }
    
    grid_search_forest = GridSearchCV(estimator = RandomForestRegressor(), param_grid = grid_para_forest,
                                      scoring='r2',
                                      cv = 5, n_jobs = -1)
    grid_search_forest.fit(final_train_df,y_train)
    r_d=grid_search_forest.predict(final_train_df)
    
    return grid_search_forest, r_d

# The Classification Model

In [8]:
def LC_Classifier(df,col_to_drop,target,**grade_weight_dict):
    '''
    df: dataframe of interest (for x_train)
    col_to_drop: accepts a list of column names (strings) that will be dropped from the model
    target: the target variable, in the format of a string
    **grade_weight_dict: the dictionary for weights for different grades
    **grade_weight_dict is in the form of:
    grade_dict={'A':1,
                'B':1,
                'C':1,
                'D':1,
                'E':1,
                'F':1,
                'G':1}
    '''
    df['weight']=df['grade'].map(grade_weight_dict)
    grade_weight_array=df['weight'].values
    index = df.index
    grade_weight_frame = pd.DataFrame(grade_weight_array, index=index)

    x_train=df.drop(col_to_drop,axis=1)
    y_train=df.loc[:,target]
    
    cat_list=columns_of_type(x_train,'string')
    cont_list=columns_of_type(x_train,'number')
    
    x_train.reset_index(drop=True,inplace=True)
    label_encoded_df=label_encode_column(x_train,cat_list)
    
#     final_train_df=label_encoded_df.apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
    final_train_df=label_encoded_df
    
    def grade_weighted_accuracy(y_true,y_pred,sample_weight):
        weighted_acc=accuracy_score(y_true,y_pred,
                                    sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1),
                                    normalize=True)
        return weighted_acc    

    score_params = {"sample_weight": grade_weight_frame}

    grade_weighted_scorer = make_scorer(score_func=grade_weighted_accuracy,
                                        greater_is_better=True,
                                        needs_proba=False,
                                        needs_threshold=False,
                                        **score_params)
    grid_para_forest = {
        'n_estimators': [50],
        'max_depth':[None],
        'max_features':['auto'],
        'min_samples_leaf':[1],
        'min_samples_split':[2]}

    grid_search_forest = GridSearchCV(estimator=RandomForestClassifier(),\
                                      param_grid=grid_para_forest,\
                                      n_jobs=-1,\
                                      scoring='accuracy',cv=5,\
                                      return_train_score=False)
    grid_search_forest.fit(final_train_df,y_train)
    sampled_prob=grid_search_forest.predict_proba(final_train_df)
    
    return grid_search_forest, sampled_prob

### adjustment of probability from classification model (to address downsampling)

In [9]:
def adjust_prob(unsampled_df,sampled_df,sampled_prob): # sampled_df, sampled_prob can also be test_df, test_prob
    # turning the predicted probability into a dataframe with column name default_prob
    sample_prob_df=pd.DataFrame(sampled_prob[:,0],columns=['sampled_prob'])
    # find actual default rate for each class
    grade_rate=unsampled_df.groupby('grade')['loan_status'].apply(lambda x:(x=='Default').sum()/x.count())
    grade_rate_dict=grade_rate.to_dict()
    # mapping the unsampled_df default rates to the test_df=sampled_df
    # and then getting the array of default_rates in the test_df
    sampled_df['default_rate']=sampled_df['grade'].map(grade_rate_dict)
    sampled_df.reset_index(drop=True, inplace=True)
    sample_prob_df.reset_index(drop=True, inplace=True)
    pre_adjust_df=pd.concat([sampled_df,sample_prob_df],axis=1)
    # Adjusting the default_probability to the true probability (accounting for down/up sampling)    
    sampled_frac=0.5
    real_prob=[]
    for row in pre_adjust_df.loc[:,['default_rate','sampled_prob']].iterrows():
        beta=sampled_frac/(1-row[1]['default_rate'])
        real_prob.append(beta*row[1]['sampled_prob']/((beta-1)*row[1]['sampled_prob']+1))
        #     prob=1/(1+(1/original_fraction-1)/(1/sampled_fraction-1)*(1/sampled_prob-1))
    a=pd.DataFrame(real_prob,columns=['actual_prob'])
    b=pd.DataFrame(sampled_prob[:,0],columns=['downsampled_prob'])
    return pd.concat([a,b],axis=1)

# Segmenting dataframe incrementally by time

In [10]:
# Defining the time windows:
timeWindows=[('2007-07','2007-12'),
             ('2008-01','2008-06'),
             ('2008-07','2008-12'),
             ('2009-01','2009-06'),
             ('2009-07','2009-12'),
             ('2010-01','2010-06'),
             ('2010-07','2010-12'),
             ('2011-01','2011-06'),
             ('2011-07','2011-12'),
             ('2012-01','2012-06'),
             ('2012-07','2012-12'),
             ('2013-01','2013-06'),
             ('2013-07','2013-12'),
             ('2014-01','2014-06'),
             ('2014-07','2014-12'),
             ('2015-01','2015-06'),
             ('2015-07','2015-12'),
             ('2016-01','2016-06'),
             ('2016-07','2016-12'),
             ('2017-01','2017-06'),
             ('2017-07','2017-12'),
             ('2018-01','2018-06'),
             ('2018-07','2018-12'),
            ]

# timeWindows=[('2007-07','2007-12'),
#              ('2008-01','2008-06'),
#              ('2008-07','2008-12')
#             ]
df.issue_d=pd.to_datetime(df.issue_d)

In [11]:
file_clf = open("pickle_model_clf.pkl",'wb')
file_reg_def = open("pickle_model_reg_def.pkl",'wb')
file_reg_paid = open("pickle_model_reg_paid.pkl",'wb')
for timeIncrement in timeWindows:
    print(timeIncrement)
    timedf=df.loc[(df.issue_d>=timeIncrement[0]) & (df.issue_d<=timeIncrement[1]),:]
    
    # Fitting with the classifier (target=loan_status) and adjusting the output probability    
    model_clf,sampled_prob=LC_Classifier(timedf,drop_list,'loan_status',grade_weight_dict=grade_weight_dict)
    prob_df=adjust_prob(pre_df,timedf,sampled_prob)
    
    # Fitting with the regressor (target=return_rate) for both default and non-default data
    df_paid=timedf[timedf.loan_status=='Fully Paid']
    df_default=timedf[timedf.loan_status=='Default']
    model_reg_paid,r_paid=LC_Regressor(df_paid,drop_list,'return_rate',grade_weight_dict=grade_weight_dict)
    model_reg_default,r_default=LC_Regressor(df_default,drop_list,'return_rate',grade_weight_dict=grade_weight_dict)
    
    pickle.dump(model_clf, file_clf)
    pickle.dump(model_reg_default, file_reg_def)    
    pickle.dump(model_reg_paid, file_reg_paid)

file_clf.close()
file_reg_def.close()
file_reg_paid.close()


('2007-07', '2007-12')


/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/auscheng/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

('2008-01', '2008-06')


/home/auscheng/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

('2008-07', '2008-12')


...

/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

('2018-01', '2018-06')


/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

('2018-07', '2018-12')


/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/auscheng/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

In [12]:
my_x,my_y=data_transform(df.sample(1000),drop_list,'loan_status',grade_weight_dict)
# my_x,my_y=data_transform(df,drop_list,'return_rate',grade_weight_dict)

In [13]:
# Save to file in the current working directory
    


In [14]:
# Load from file
with open('pickle_model_clf.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
    
# Calculate the accuracy score and predict target values
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(my_x)

Test score: 51.70 %


In [15]:
with open('pickle_model_clf.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
    pickle_model2 = pickle.load(file)
    pickle_model3 = pickle.load(file)
    pickle_model4 = pickle.load(file)

# Calculate the accuracy score and predict target values
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(my_x)

Test score: 51.70 %


In [16]:
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model2.predict(my_x)

Test score: 51.70 %


In [17]:
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model3.predict(my_x)

Test score: 51.70 %


In [18]:
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model4.predict(my_x)

Test score: 51.70 %
